In [2]:
#这里和readme代码一样，确定读过readme了吗

%matplotlib qt5
import classes
import importlib

importlib.reload(classes)

from classes.trend import Trend
from classes.kline_client import Kline_client
from classes.timepoint import Timepoint
from classes.plot_driver import Plot_driver
from classes.analyse_area import Analyse_area

In [25]:
kline_clt = Kline_client()

In [182]:
#在这里创建一个工作区对象，里面包含很多趋势，并对趋势进行合并
# del te
te = Analyse_area(Timepoint(2022,3,5,12,31),Timepoint(2022,3,6,12,31),kline_clt)
te.synchronize(Timepoint(2022,3,12,12,31))

In [186]:
#创建绘图助理，可以把工作区中的趋势画出来，layer控制从最大的趋势往细处精细的层数
pds = Plot_driver(kline_clt)
pds.draw(te,layer = 5)

c:\Users\ASUS\anaconda3\Lib\site-packages\mplfinance\_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


In [111]:
begin = Timepoint(2022,3,5,12,31)
point = begin.get_after(1)
te = Analyse_area(begin,point,kline_clt)

In [181]:
point.next()
te.synchronize(point)
pds.draw(te,layer = 1)

c:\Users\ASUS\anaconda3\Lib\site-packages\mplfinance\_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+
